In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from __future__ import division
import os
import re
import sys
import random
import time
import binascii
from bisect import bisect_right
from heapq import heappop, heappush
import pandas as pd
import numpy as np
from tabulate import tabulate
from itertools import combinations
#from beautifultable import BeautifulTable

# This is the number of components in the resulting MinHash signatures.
numHashes = 100


# It ships with data set sizes 100, 1000, 2500, and 10000.
numDocs = 100
trainFile = "/content/drive/MyDrive/Colab Notebooks/data/articles_" + str(numDocs) + ".train.txt"
testFile = "/content/drive/MyDrive/Colab Notebooks/data/articles_" + str(numDocs) + ".test.txt"

with open('/content/drive/MyDrive/Colab Notebooks/data/file3.txt', 'w', encoding="utf-8") as dataFile:
  
    # Iterate through list
    for names in [trainFile, testFile]:
  
        # Open each file in read mode
        with open(names, encoding="utf-8") as infile:
  
            # read the data from file1 and file2 and write it in file3
            dataFile.write(infile.read())
        dataFile.write("\n")


#truthFile = "./data/articles_" + str(numDocs) + ".truth.txt"


# =============================================================================
#                     LSH techniques started from here
# =============================================================================


#Convert Documents To Sets of Shingles

print ("\n\t...Shingling articles... ")

# The current shingle ID value to assign to the next new shingle we encounter.
curShingleID = 0

# Create a dictionary of the articles, mapping the article identifier (e.g.,"t8470") to the list 
#of shingle IDs that appear in the document.
docsAsShingleSets = {};
  

f = open('/content/drive/MyDrive/Colab Notebooks/data/file3.txt', "r" , encoding="utf8")

docNames = []

t0 = time.time()

totalShingles = 0

docNM = []

for i in range(0, numDocs):
  
  words = f.readline().split(" ")  
  
  docID = words[0]
  
  # Maintain a list of all document IDs.  
  docNames.append(docID)
    
  del words[0]  
  
  shinglesInDoc = set()
  
  # For each word in the document...
  for index in range(0, len(words) - 2):

    #shingle = list(words[index].union(words[index + 1]).union(words[index + 2]))
    shingle = words[index] + " " + words[index + 1] + " " + words[index + 2]
    #print(shingle)
    #print("\n")
    

    # Hash the shingle to a 32-bit integer.
    crc = binascii.crc32(shingle.encode('utf8')) & 0xffffffff
    
    # Add the hash value to the list of shingles for the current document.  
    shinglesInDoc.add(crc)
  
  docsAsShingleSets[docID] = shinglesInDoc
  #print (shinglesInDoc)
  #print("\n")
  
  # Count the number of shingles across all documents.
  totalShingles = totalShingles + (len(words) - 2)

f.close()  



# Report how long shingling took.
print ("\nShingling " + str(numDocs) + " docs took %.2f sec." % (time.time() - t0))
 
print ('\nAverage shingles per doc: %.2f' % (totalShingles / numDocs))

# =============================================================================
#                     Define Triangle Matrices
# =============================================================================

# Calculate the number of elements needed in our triangle matrix
numElems = int(numDocs * (numDocs - 1) / 2)

# Initialize two empty lists to store the similarity values
#for the actual Jaccard Similarity values
JSim = [0 for x in range(numElems)]
#for the estimated Jaccard Similarities found by comparing the MinHash signatures.
estJSim = [0 for x in range(numElems)]



# Define a function to map a 2D matrix coordinate into a 1D index.
def getTriangleIndex(i, j):
  # If i == j that's an error.
  if i == j:
    sys.stderr.write("Can't access triangle matrix with i == j")
    sys.exit(1)
  # If j < i just swap the values.
  if j < i:
    temp = i
    i = j
    j = temp
  
  # Calculate the index within the triangular array.
  # This fancy indexing scheme is taken from pg. 211 of:
  # http://infolab.stanford.edu/~ullman/mmds/ch6.pdf
  # But I adapted it for a 0-based index.
  # Note: The division by two should not truncate, it
  #       needs to be a float. 
  k = int(i * (numDocs - (i + 1) / 2.0) + j - i) - 1
  
  return k

        
# =============================================================================
#                 Generate MinHash Signatures
# =============================================================================

# Time this step.
#t0 = time.time()

#print ("\nGenerating random hash functions...")

# Record the maximum shingle ID that we assigned.
maxShingleID = 2**32-1

# We need the next largest prime number above 'maxShingleID'.
# I looked this value up here: 
# http://compoasso.free.fr/primelistweb/page/prime/liste_online_en.php
nextPrime = 4294967311


# Our random hash function will take the form of:
#   h(x) = (a*x + b) % c
# Where 'x' is the input value, 
#'a' and 'b' are random coefficients, 
#and 'c' is a prime number just greater than maxShingleID.

# Generate a list of 'k' random coefficients for the random hash functions
def pickRandomCoeffs(k):
  # Create a list of 'k' random values.
  randList = []
  
  while k > 0:
    # Get a random shingle ID.
    randIndex = random.randint(0, maxShingleID) 
  
    # Ensure that each random number is unique.
    while randIndex in randList:
      randIndex = random.randint(0, maxShingleID) 
    
    # Add the random number to the list.
    randList.append(randIndex)
    k = k - 1
    
  return randList

# For each of the 'numHashes' hash functions, generate a different coefficient 'a' and 'b'.   
coeffA = pickRandomCoeffs(numHashes)
coeffB = pickRandomCoeffs(numHashes)

print ("\n\n\t...Generating MinHash signatures for all documents...")

# List of documents represented as signature vectors
signatures = []

# For each document...
for docID in docNames:
 
  shingleIDSet = docsAsShingleSets[docID]

  signature = []
  
  # For each of the random hash functions...
  for i in range(0, numHashes):
    
    # For each of the shingles actually in the document, calculate its hash code using hash function 'i'. 
    minHashCode = nextPrime + 1
    
    # For each shingle in the document...
    for shingleID in shingleIDSet:
      # Evaluate the hash function.
      hashCode = (coeffA[i] * shingleID + coeffB[i]) % nextPrime 
      
      # Track the lowest hash code seen.
      if hashCode < minHashCode:
        minHashCode = hashCode

    # Add the smallest hash code value as component number 'i' of the signature.
    signature.append(minHashCode)
    # Calculate the elapsed time (in seconds)
  #print(signature)
  
  #Store the MinHash signature for this document.
  signatures.append(signature)

#print(signatures[:5])
#print("\n")
# Calculate the elapsed time (in seconds)
elapsed1 = (time.time() - t0)
print ("\nGenerating MinHash signatures for ", numHashes, " number of permutations took %.2fsec" % elapsed1)  

# =============================================================================
#                     Compare All Signatures
# =============================================================================  

print ("\n\n\t...Comparing all signatures...")  
  
# N x N matrix
#t0 = time.time()

# For each of the test documents...
for i in range(0, numDocs):
  # Get the MinHash signature for document i.
  signature1 = signatures[i]
    
  # For each of the other test documents...
  for j in range(i + 1, numDocs):
    
    # Get the MinHash signature for document j.
    signature2 = signatures[j]
    
    count = 0
    # Count the number of positions in the minhash signature which are equal.
    for k in range(0, numHashes):
      count = count + (signature1[k] == signature2[k])
    
    # Record the percentage of positions which matched.    
    estJSim[getTriangleIndex(i, j)] = (count / numHashes)


    
    
# =============================================================================
#                   Display Similar Document Pairs
# =============================================================================  

# Count the true positives and false positives.
data = []
tp = 0
fp = 0

col_names = ["Doc_1", "Doc_2","Estimated J","Actual J","Distance"]
  
threshold = 0.8  
print ("\nList of Document Pairs with J(d1,d2) more than ", threshold, "is considered as a similarity\n")

#print ("                   Est. J   Act. J")


# For each of the document pairs...
for i in range(0, numDocs):  
  for j in range(i + 1, numDocs):
    # Retrieve the estimated similarity value for this pair.
    estJ = estJSim[getTriangleIndex(i, j)]
    
    
    if estJ > threshold:
    
      # Calculate the actual Jaccard similarity for validation.
      s1 = docsAsShingleSets[docNames[i]]
      s2 = docsAsShingleSets[docNames[j]]
      J = (len(s1.intersection(s2)) / len(s1.union(s2)))
      distance = abs(float(estJ - J))
      
      data += [[docNames[i], docNames[j], estJ,  J, distance]]

print(tabulate(data, headers=col_names, tablefmt="fancy_grid"))

# Calculate the elapsed time (in seconds)
elapsed2 = (time.time() - t0)
print("\nWhole process took %.2f seconds" % elapsed2)
print ("\n\n\tFor ", numHashes, " number of permutations, MinHash technique took %.2f seconds" % (elapsed2 - elapsed1), "for comparing all signatures")

#table.append_row([docNames[i],  docNames[j],  estJ,  J])
#print(table)
#display table



	...Shingling articles... 

Shingling 100 docs took 0.03 sec.

Average shingles per doc: 243.69


	...Generating MinHash signatures for all documents...

Generating MinHash signatures for  100  number of permutations took 1.32sec


	...Comparing all signatures...

List of Document Pairs with J(d1,d2) more than  0.8 is considered as a similarity

╒═════════╤═════════╤═══════════════╤════════════╤═════════════╕
│ Doc_1   │ Doc_2   │   Estimated J │   Actual J │    Distance │
╞═════════╪═════════╪═══════════════╪════════════╪═════════════╡
│ t980    │ t2023   │          0.97 │   0.979167 │ 0.00916667  │
├─────────┼─────────┼───────────────┼────────────┼─────────────┤
│ t9536   │ t9537   │          0.99 │   0.975    │ 0.015       │
├─────────┼─────────┼───────────────┼────────────┼─────────────┤
│ t1088   │ t5015   │          0.99 │   0.980545 │ 0.00945525  │
├─────────┼─────────┼───────────────┼────────────┼─────────────┤
│ t1297   │ t4638   │          0.98 │   0.98062  │ 0.000620155 │
├